In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('large_dataset.csv')

In [3]:
df.shape

(30000000, 5)

In [4]:
df.head()

,f1,f2,f3,f4,target
0,0.203936,-0.291191,-0.617251,-1.461300,10.796660
1,-1.195432,0.069214,0.291861,1.972511,11.154018
2,-2.560271,-0.827846,0.405551,-0.132588,-20.342684
3,-1.131196,0.252654,0.961175,1.236873,1.407638
4,0.858774,-0.461605,1.095498,-0.871580,-17.043060


In [5]:
df.describe()

,f1,f2,f3,f4,target
count,3.000000e+07,3.000000e+07,3.000000e+07,3.000000e+07,3.000000e+07
mean,-3.258116e-04,-1.060974e-04,-1.360234e-05,-6.534420e-05,-8.363976e-03
std,9.999900e-01,1.000074e+00,9.998615e-01,1.000105e+00,3.131135e+01
min,-5.491886e+00,-5.333572e+00,-5.312364e+00,-5.237933e+00,-1.694348e+02
25%,-6.748877e-01,-6.744943e-01,-6.743856e-01,-6.744510e-01,-2.112416e+01
50%,-3.116059e-04,-2.164725e-04,-4.358653e-05,-2.934173e-04,-1.129371e-02
75%,6.741835e-01,6.743107e-01,6.744813e-01,6.746014e-01,2.111568e+01
max,5.500586e+00,5.449671e+00,5.500811e+00,5.347441e+00,1.778399e+02


In [6]:
!pip install vaex

In [7]:
import vaex

In [8]:
vaex_df = vaex.from_pandas(df)

In [9]:
type(vaex_df)

vaex.dataframe.DataFrameLocal

In [10]:
vaex_df.export_hdf5('large_data.hdf5')

In [11]:
vaex_df = vaex.open('large_data.hdf5')

In [12]:
vaex_df.head()

#,f1,f2,f3,f4,target
0,0.203936,-0.291191,-0.617251,-1.4613,10.7967
1,-1.19543,0.0692141,0.291861,1.97251,11.154
2,-2.56027,-0.827846,0.405551,-0.132588,-20.3427
3,-1.1312,0.252654,0.961175,1.23687,1.40764
4,0.858774,-0.461605,1.0955,-0.87158,-17.0431
5,0.63273,2.07437,-0.700978,1.36085,49.2754
6,-1.92771,0.155743,-0.394158,-1.3712,11.0902
7,-0.165462,-0.481113,-1.29813,1.75204,-41.1136
8,-1.31883,0.393728,-1.83311,-0.599728,1.87724
9,-0.671377,-0.172073,-0.75585,1.05745,-26.2623


In [14]:
vaex_df = vaex_df.shuffle()
df_train, df_test = vaex_df.ml.train_test_split(test_size=0.2)

C:\Users\91842\anaconda3\lib\site-packages\vaex\ml\__init__.py:31: UserWarning: Make sure the DataFrame is shuffled
  warnings.warn('Make sure the DataFrame is shuffled')


In [15]:
df_train.shape

(24000000, 5)

In [16]:
df_test.shape

(6000000, 5)

In [17]:
from vaex.ml.sklearn import IncrementalPredictor
from sklearn.linear_model import SGDRegressor

features = ['f1','f2','f3','f4']
target = 'target'

model = SGDRegressor()

vaex_model = IncrementalPredictor(features=features,target=target,model=model,batch_size=500000)

vaex_model.fit(df=df_train,progress='widget')

In [19]:
df_test = vaex_model.transform(df_test)

In [20]:
df_test.head()

#,f1,f2,f3,f4,target,prediction
0,0.651098,1.75828,-1.55602,0.205312,52.0572,41.9285
1,-0.930079,0.66736,1.38524,-0.785785,17.9182,16.1285
2,-0.757777,-1.0052,1.03499,0.227977,-37.9689,-24.1394
3,0.320047,1.03793,1.27838,2.78425,-7.66551,25.0803
4,-1.57124,-0.383367,0.162823,-0.70573,-13.792,-9.57404
5,-1.25979,-0.777414,0.224138,-0.925356,21.0758,-18.9242
6,-0.0639199,-0.143284,-0.103907,-0.761464,-9.42623,-3.48351
7,0.798208,0.53451,-1.62919,-1.36567,-8.41418,12.6728
8,0.465624,-0.659685,1.04316,0.305888,-25.7486,-15.5313
9,-0.749915,-0.151375,-0.742794,-0.0452924,20.3753,-4.0625


In [21]:
from sklearn.metrics import r2_score,mean_absolute_error
print(r2_score(df_test['target'].values,df_test['prediction'].values))
print(mean_absolute_error(df_test['target'].values,df_test['prediction'].values))

0.5920774016319219
15.96033360781515
